In [5]:
import ffmpegcv

vidin = ffmpegcv.VideoCaptureNV('input.mp4', gpu=1)
vidout = ffmpegcv.VideoWriterNV('output.mp4', 'h264', vidin.fps)  #NVIDIA-GPU

with vidin, vidout:
    for frame in vidin:
        #cv2.imshow('image', frame)
        vidout.write(frame)

In [5]:
import torch
print(torch.__version__)  # Should show 2.x.x
print(torch.cuda.is_available())  # Should return True if GPU is available
print(dir(torch))  # Should show 'load' in the list

AttributeError: module 'torch' has no attribute '__version__'

In [4]:
import ffmpegcv
import time
from pynvml import *
from contextlib import contextmanager

@contextmanager
def gpu_monitoring():
    """Context manager for GPU monitoring"""
    class GPUMonitor:
        def __init__(self, handle):
            self.handle = handle
            
        def get_stats(self):
            try:
                util = nvmlDeviceGetUtilizationRates(self.handle)
                mem = nvmlDeviceGetMemoryInfo(self.handle)
                return {
                    'gpu_util': util.gpu,
                    'mem_util': util.memory,
                    'mem_used': mem.used / 1024**2  # MB
                }
            except NVMLError as e:
                print(f"GPU Stats Error: {e}")
                return {'gpu_util': -1, 'mem_util': -1, 'mem_used': -1}
    
    handle = None
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        yield GPUMonitor(handle)
    except NVMLError as e:
        print(f"NVML Init Error: {e}")
        yield None
    finally:
        if handle is not None:
            try:
                nvmlShutdown()
            except NVMLError as e:
                print(f"NVML Shutdown Error: {e}")

# Video processing
vidin = ffmpegcv.VideoCaptureNV('input.mp4', gpu=0)
vidout = ffmpegcv.VideoWriterNV('output.mp4', 'h264', vidin.fps, gpu=0)

print(f"Processing on GPU 0 | {vidin.width}x{vidin.height} @ {vidin.fps:.1f} FPS")
print("Frame | GPU% | MEM% | Used(MB) | FPS")
print("------------------------------------")

start_time = time.time()
frame_count = 0

try:
    with gpu_monitoring() as monitor, vidin, vidout:
        for frame in vidin:
            vidout.write(frame)
            frame_count += 1
            
            if frame_count % 30 == 0:
                stats = monitor.get_stats() if monitor else {'gpu_util': -1, 'mem_util': -1, 'mem_used': -1}
                elapsed = max(0.001, time.time() - start_time)
                fps = frame_count / elapsed
                
                print(
                    f"{frame_count:5d} | "
                    f"{stats['gpu_util']:3d}% | "
                    f"{stats['mem_util']:3d}% | "
                    f"{stats['mem_used']:7.1f} | "
                    f"{fps:5.1f}"
                )
                
except KeyboardInterrupt:
    print("\nProcess interrupted")
finally:
    total_time = time.time() - start_time
    print("\nSummary:")
    print(f"Frames processed: {frame_count}")
    print(f"Total time: {total_time:.1f}s")
    print(f"Average FPS: {frame_count/total_time:.1f}")

Processing on GPU 0 | 3840x2160 @ 30.0 FPS
Frame | GPU% | MEM% | Used(MB) | FPS
------------------------------------
   30 |   3% |   1% |  1139.8 |   7.5
   60 |   1% |   1% |  1139.8 |  10.0
   90 |   2% |   1% |  1139.8 |  11.1
  120 |   3% |   2% |  1139.8 |  11.7
  150 |   1% |   1% |  1139.8 |  12.2
  180 |   1% |   1% |  1139.8 |  12.6
  210 |   1% |   1% |  1139.8 |  12.8

Summary:
Frames processed: 231
Total time: 18.6s
Average FPS: 12.5
